In [ ]:
# import necessary libraries

%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import matplotlib
#matplotlib.use('TkAgg')
import matplotlib.pyplot as plt
import pandas as pd
import statsmodels.api as sm

# BigQuery settings
from google.cloud import bigquery
from google.cloud.bigquery import dbapi;
client = bigquery.Client("mining-clinical-decisions"); # Project identifier
conn = dbapi.connect(client);
cursor = conn.cursor();
import google.auth

credentials, your_project_id = google.auth.default( scopes=["https://www.googleapis.com/auth/cloud-platform"])
bqclient = bigquery.Client(credentials=credentials, project="mining-clinical-decisions",)



In [ ]:
df_list=[]

### Unique Workstations
  
 

Of note. I did not change TPAP since it really didn,t matter what we called that table, so I left it as it was. All other mentions to TPA were changed from old code.

In [ ]:
query = '''
WITH TPAP_ACTUAL_TIMES AS
(
SELECT 
    AL.pat_enc_csn_id_coded, 
    min(DATETIME_SUB(AL.cathTime, INTERVAL TMP.JITTER DAY)) as cath_time,
    min(DATETIME_SUB(AL.emergencyAdmitTime, INTERVAL TMP.JITTER DAY)) as emergency_admit_time, 
    min(DATETIME_SUB(DATETIME_SUB(AL.cathTime, INTERVAL TMP.JITTER DAY), INTERVAL 1 HOUR)) as tw_start_time,
    min(DATETIME_ADD(DATETIME_SUB(AL.cathTime, INTERVAL TMP.JITTER DAY), INTERVAL 1 HOUR)) as tw_end_time,

FROM `rose_team.cohort_AL` as AL,
  `rose_team.tmp` as TMP
WHERE AL.rit_uid = TMP.ANON_ID
GROUP BY AL.pat_enc_csn_id_coded
ORDER BY cath_time
),

PROVIDER_MAPPING AS
(
SELECT pat_enc_csn_id_coded, user_deid AS provider_id, emergencyAdmitTime, cathTime, count(*) AS even_count
  FROM `rose_team.cohort_AL` as AL
  WHERE 
    metric_name like "%Lab%" OR metric_name like "% lab%"
    OR metric_name like "%Encounter%" OR metric_name like "%encounter%"
    OR metric_name like "%Flowchart%" OR metric_name like "%flowchart%"
    OR metric_name like "%Order%" OR metric_name like "%order%"   
    OR metric_name like "%Result%" OR metric_name like "%result%" 
    OR metric_name like "%Medications%" OR metric_name like "%medications%"
    OR metric_name like "%MAR%" 
    OR metric_name like "%Note%" OR metric_name like "%note%"
    OR metric_name like "%History%" OR metric_name like "%history%"
    OR metric_name like "%Imag%" OR metric_name like "%imag%"

  
  GROUP BY pat_enc_csn_id_coded, user_deid,emergencyAdmitTime, cathTime
),

TT AS
(
SELECT 
  *
FROM 
  TPAP_ACTUAL_TIMES
INNER JOIN
  PROVIDER_MAPPING
USING 
  (pat_enc_csn_id_coded)
),


AL_ALL_ACTUAL_TIMES AS
(SELECT AL.*, DATETIME_SUB(AL.access_time_jittered, INTERVAL TMP.JITTER DAY) as access_time_real
FROM `rose_team.shc_access_log_de_dep_id` as AL,
`rose_team.tmp` as TMP
WHERE AL.rit_uid=TMP.ANON_ID
    AND metric_name NOT LIKE 'Inpatient Patient Lists activity accessed'
ORDER BY AL.rit_uid
),

ALL_TT_ENCOUNTERS AS (

SELECT 
  TT.*, access_time_real, metric_name, workstation_id
FROM 
  TT
INNER JOIN
  AL_ALL_ACTUAL_TIMES
ON
  (AL_ALL_ACTUAL_TIMES.user_deid = TT.provider_id)
AND
  access_time_real BETWEEN tw_start_time AND tw_end_time
),

ALL_TT_METRICS AS (

SELECT 
    pat_enc_csn_id_coded,
    cath_time,
    emergency_admit_time,
    provider_id,
    COUNT(DISTINCT workstation_id) AS num_unq_wrkstn
FROM 
    ALL_TT_ENCOUNTERS
GROUP BY 1,2,3,4
)

SELECT 
    pat_enc_csn_id_coded,
    cath_time,
    emergency_admit_time,
    AVG(num_unq_wrkstn) AS num_unq_wrkstn
FROM 
    ALL_TT_METRICS
GROUP BY 
    1,2,3 
    ORDER BY 
    cath_time
'''



In [ ]:
dataframe = bqclient.query(query).result().to_dataframe()
print(dataframe)
dataframe= dataframe[['pat_enc_csn_id_coded', 'num_unq_wrkstn']]
print(dataframe.head())
df_list.append(dataframe)

### Task Switching

In [ ]:
query = '''
WITH TPAP_ACTUAL_TIMES AS
(
SELECT
AL.rit_uid,
AL.pat_enc_csn_id_coded,
min(DATETIME_SUB(AL.cathTime, INTERVAL TMP.JITTER DAY)) as cath_time,
min(DATETIME_SUB(AL.emergencyAdmitTime, INTERVAL TMP.JITTER DAY)) as emergency_admit_time,
min(DATETIME_SUB(DATETIME_SUB(AL.cathTime, INTERVAL TMP.JITTER DAY), INTERVAL 1 HOUR)) as tw_start_time,
min(DATETIME_ADD(DATETIME_SUB(AL.cathTime, INTERVAL TMP.JITTER DAY), INTERVAL 1 HOUR)) as tw_end_time,
FROM `rose_team.cohort_AL` as AL,
`rose_team.tmp` as TMP
WHERE AL.rit_uid = TMP.ANON_ID
GROUP BY AL.rit_uid, AL.pat_enc_csn_id_coded
ORDER BY cath_time
),
PROVIDER_MAPPING AS
(
SELECT pat_enc_csn_id_coded, user_deid AS provider_id, emergencyAdmitTime, cathTime, count(*) AS even_count
FROM `rose_team.cohort_AL` as AL
WHERE
(metric_name like "%Lab%" OR metric_name like "% lab%"
OR metric_name like "%Encounter%" OR metric_name like "%encounter%"
OR metric_name like "%Flowchart%" OR metric_name like "%flowchart%"
OR metric_name like "%Order%" OR metric_name like "%order%"
OR metric_name like "%Result%" OR metric_name like "%result%"
OR metric_name like "%Medications%" OR metric_name like "%medications%"
OR metric_name like "%MAR%"
OR metric_name like "%Note%" OR metric_name like "%note%"
OR metric_name like "%History%" OR metric_name like "%history%"
OR metric_name like "%Imag%" OR metric_name like "%imag%"
)
GROUP BY pat_enc_csn_id_coded, user_deid, emergencyAdmitTime, cathTime
),
TT AS
(
SELECT
*
FROM
TPAP_ACTUAL_TIMES
INNER JOIN
PROVIDER_MAPPING
USING
(pat_enc_csn_id_coded)
),
-- Generate AL with actual times
AL_ALL_ACTUAL_TIMES AS
(SELECT AL.*, DATETIME_SUB(AL.access_time_jittered, INTERVAL TMP.JITTER DAY) as access_time_real
FROM `rose_team.shc_access_log_de_dep_id` as AL,
`rose_team.tmp` as TMP
WHERE
AL.rit_uid=TMP.ANON_ID
AND metric_name NOT LIKE 'Inpatient Patient Lists activity accessed'
ORDER BY AL.rit_uid
),
FILTERED_EVENTS AS (
SELECT
TT.*,
AL_ALL_ACTUAL_TIMES.access_time_real,
AL_ALL_ACTUAL_TIMES.metric_name,
AL_ALL_ACTUAL_TIMES.workstation_id,
AL_ALL_ACTUAL_TIMES.rit_uid AS patient_id,
LEAD(AL_ALL_ACTUAL_TIMES.rit_uid)
OVER (PARTITION BY pat_enc_csn_id_coded, provider_id ORDER BY access_time_real ASC) AS next_patient_id
FROM
TT
INNER JOIN
AL_ALL_ACTUAL_TIMES
ON
AL_ALL_ACTUAL_TIMES.user_deid = TT.provider_id
WHERE
access_time_real BETWEEN tw_start_time AND tw_end_time
ORDER BY
pat_enc_csn_id_coded, provider_id, access_time_real
),
TASK_SWITCHING AS
(
SELECT
*,
(CASE WHEN patient_id = next_patient_id THEN 0 ELSE 1 END) AS task_switched
FROM FILTERED_EVENTS
),
PROVIDER_LEVEL_SWITCHING AS
(
SELECT
DATETIME_DIFF(cath_time, emergency_admit_time, MINUTE) AS cath_min,
pat_enc_csn_id_coded,
provider_id,
SUM(task_switched) AS total_switching,
COUNT(distinct(patient_id))-1 AS view_count
FROM TASK_SWITCHING
GROUP BY 1, pat_enc_csn_id_coded, provider_id
)

SELECT 
    pat_enc_csn_id_coded,
    cath_min,
    AVG(total_switching) AS avg_task_switching,
    AVG(view_count) AS view_count
FROM 
    PROVIDER_LEVEL_SWITCHING
GROUP BY 
    1,2
'''

Update April 15th, 2022: 'View count' removed from the dataframe that we are adding to DF list

In [ ]:
dataframe = bqclient.query(query).result().to_dataframe()
dataframe= dataframe[['pat_enc_csn_id_coded', 'cath_min', 'avg_task_switching']]
print(dataframe.head())
df_list.append(dataframe)

In [ ]:
print(dataframe)

### Chart views
Robert Edits: Changing this so that we are measuring the number of distinct views of other patients, not the total number of views of other patient's "%encounter%" metrics.
As of January 14th - Trying to determine why there are 252 unique CSN's in one table versus 243 in another.
Update: April 15th, 236 chart views versus 228 actual patients.
Robert searched the code and found this table had a unique event "reports" which was added to the definiton to find providers. Deleted that line.

In [ ]:
query='''
WITH TPAP_ACTUAL_TIMES AS
(
SELECT 
    AL.rit_uid,
    AL.pat_enc_csn_id_coded, 
    min(DATETIME_SUB(AL.cathTime, INTERVAL TMP.JITTER DAY)) as cath_time,
    min(DATETIME_SUB(AL.emergencyAdmitTime, INTERVAL TMP.JITTER DAY)) as emergency_admit_time, 
    min(DATETIME_SUB(DATETIME_SUB(AL.cathTime, INTERVAL TMP.JITTER DAY), INTERVAL 1 HOUR)) as tw_start_time,
    min(DATETIME_ADD(DATETIME_SUB(AL.cathTime, INTERVAL TMP.JITTER DAY), INTERVAL 1 HOUR)) as tw_end_time,
FROM `rose_team.cohort_AL` as AL,
  `rose_team.tmp` as TMP
WHERE AL.rit_uid = TMP.ANON_ID
GROUP BY AL.rit_uid, AL.pat_enc_csn_id_coded
ORDER BY cath_time
),
PROVIDER_MAPPING AS
(
SELECT pat_enc_csn_id_coded, user_deid AS provider_id, emergencyAdmitTime, cathTime, count(*) AS even_count
  FROM `rose_team.cohort_AL` as AL
  WHERE 
    (metric_name like "%Lab%" OR metric_name like "% lab%"
    OR metric_name like "%Encounter%" OR metric_name like "%encounter%"
    OR metric_name like "%Flowchart%" OR metric_name like "%flowchart%"
    OR metric_name like "%Order%" OR metric_name like "%order%"    
    OR metric_name like "%Medications%" OR metric_name like "%medications%"
    OR metric_name like "%MAR%" 
    OR metric_name like "%Note%" OR metric_name like "%note%"
    OR metric_name like "%History%" OR metric_name like "%history%"
    OR metric_name like "%Imag%" OR metric_name like "%imag%"
    )
  
  GROUP BY pat_enc_csn_id_coded, user_deid,emergencyAdmitTime, cathTime
),
TT AS
(
SELECT 
  *
FROM 
  TPAP_ACTUAL_TIMES
INNER JOIN
  PROVIDER_MAPPING
USING 
  (pat_enc_csn_id_coded)
),
  -- Generate AL with actual times where metric name is Encounter%. 
  -- Every chart view should have at least one Encounter metric name.
  -- For the MOORE work, we do not want this restriction. Removing it.
AL_ALL_ACTUAL_TIMES AS
(SELECT AL.*, DATETIME_SUB(AL.access_time_jittered, INTERVAL TMP.JITTER DAY) as access_time_real
FROM `rose_team.shc_access_log_de_dep_id` as AL,
`rose_team.tmp` as TMP
WHERE 
    AL.rit_uid=TMP.ANON_ID
ORDER BY AL.rit_uid
),
FILTERED_EVENTS AS (
SELECT 
  TT.*, 
  TT.rit_uid AS INDEX_UID,
  AL_ALL_ACTUAL_TIMES.access_time_real, 
  AL_ALL_ACTUAL_TIMES.metric_name, 
  AL_ALL_ACTUAL_TIMES.workstation_id,
  AL_ALL_ACTUAL_TIMES.rit_uid AS other_uid  
FROM 
  TT
INNER JOIN
  AL_ALL_ACTUAL_TIMES
ON
    AL_ALL_ACTUAL_TIMES.user_deid = TT.provider_id
WHERE
  access_time_real BETWEEN tw_start_time AND tw_end_time
ORDER BY 
    pat_enc_csn_id_coded, provider_id, access_time_real
),
-- Subtract 1 from the counts since we don't want to include the index patient.
PROVIDER_LEVEL_METRIC AS
(
SELECT 
    -- DATETIME_DIFF(cath_time, emergency_admit_time, MINUTE) AS cath_min,
    pat_enc_csn_id_coded, 
    provider_id, 
    COUNT(DISTINCT(other_uid)) - 1 AS view_count 
    FROM FILTERED_EVENTS
GROUP BY pat_enc_csn_id_coded, provider_id
)

SELECT pat_enc_csn_id_coded, AVG(view_count) AS view_count
FROM PROVIDER_LEVEL_METRIC
GROUP BY 1

'''


In [ ]:
dataframe = bqclient.query(query).result().to_dataframe()
dataframe= dataframe[['pat_enc_csn_id_coded', 'view_count']]
print(dataframe)
df_list.append(dataframe)

In [ ]:
#bad_df[bad_df.pat_enc_csn_id_coded.isin(dataframe.pat_enc_csn_id_coded) == False]

### Days Since Last TPA Case

In [ ]:
query='''
WITH TPAP_ACTUAL_TIMES AS
(
SELECT 
    AL.pat_enc_csn_id_coded, 
    min(DATETIME_SUB(AL.cathTime, INTERVAL TMP.JITTER DAY)) as cath_time,
    min(DATETIME_SUB(AL.emergencyAdmitTime, INTERVAL TMP.JITTER DAY)) as emergency_admit_time, 
    min(DATETIME_SUB(DATETIME_SUB(AL.cathTime, INTERVAL TMP.JITTER DAY), INTERVAL 1 HOUR)) as tw_start_time,
    min(DATETIME_ADD(DATETIME_SUB(AL.cathTime, INTERVAL TMP.JITTER DAY), INTERVAL 1 HOUR)) as tw_end_time,

FROM `rose_team.cohort_AL` as AL,
  `rose_team.tmp` as TMP
WHERE AL.rit_uid = TMP.ANON_ID
GROUP BY AL.pat_enc_csn_id_coded
ORDER BY cath_time
),

PROVIDER_MAPPING AS
(
SELECT pat_enc_csn_id_coded, user_deid AS provider_id, emergencyAdmitTime, cathTime, count(*) AS even_count
  FROM `rose_team.cohort_AL` as AL
  WHERE 
    metric_name like "%Lab%" OR metric_name like "% lab%"
    OR metric_name like "%Encounter%" OR metric_name like "%encounter%"
    OR metric_name like "%Flowchart%" OR metric_name like "%flowchart%"
    OR metric_name like "%Order%" OR metric_name like "%order%"   
    OR metric_name like "%Result%" OR metric_name like "%result%" 
    OR metric_name like "%Medications%" OR metric_name like "%medications%"
    OR metric_name like "%MAR%" 
    OR metric_name like "%Note%" OR metric_name like "%note%"
    OR metric_name like "%History%" OR metric_name like "%history%"
    OR metric_name like "%Imag%" OR metric_name like "%imag%"

  
  GROUP BY pat_enc_csn_id_coded, user_deid,emergencyAdmitTime, cathTime
),

TT AS
(
SELECT 
  *
FROM 
  TPAP_ACTUAL_TIMES
INNER JOIN
  PROVIDER_MAPPING
USING 
  (pat_enc_csn_id_coded)
),

  -- Generate AL with actual times
AL_ALL_ACTUAL_TIMES AS
(SELECT AL.*, DATETIME_SUB(AL.access_time_jittered, INTERVAL TMP.JITTER DAY) as access_time_real
FROM `rose_team.shc_access_log_de_dep_id` as AL,
`rose_team.tmp` as TMP
WHERE AL.rit_uid=TMP.ANON_ID
    AND metric_name NOT LIKE 'Inpatient Patient Lists activity accessed'
ORDER BY AL.rit_uid
),

ALL_TT_ENCOUNTERS AS (

SELECT 
  TT.*, access_time_real, metric_name, workstation_id
FROM 
  TT
INNER JOIN
  AL_ALL_ACTUAL_TIMES
ON
  (AL_ALL_ACTUAL_TIMES.user_deid = TT.provider_id)
AND
  access_time_real BETWEEN tw_start_time AND tw_end_time
),

ALL_TT_METRICS AS (

SELECT 
    pat_enc_csn_id_coded,
    cath_time,
    emergency_admit_time,
    provider_id,
    DATETIME_DIFF(cath_time, LAG(emergency_admit_time) -- ,1,DATETIME(2000,1,1,0,0,0)
    OVER (PARTITION BY provider_id ORDER BY emergency_admit_time ASC), DAY) AS day_since_last_cath
FROM 
    ALL_TT_ENCOUNTERS
GROUP BY 1,2,3,4
ORDER BY emergency_admit_time
),

-- Changing this so that the default for missing values is not 30, but is null.
ALL_TT_METRICS_2 AS (

SELECT 
    pat_enc_csn_id_coded,
    cath_time,
    emergency_admit_time,
    provider_id,
    (CASE WHEN day_since_last_cath IS NOT NULL THEN day_since_last_cath ELSE NULL END) AS day_since_last_cath
FROM 
    ALL_TT_METRICS
),
-- REMOVE ANY ENCOUNTERS WITHIN THE 1st 30 days of the study period so that we have a bit of a washout period to
-- accumlate strokes cases for providers.
FIND_EARLIEST_ENC AS (
SELECT 
    DATETIME_ADD(MIN(cath_time), INTERVAL 30 DAY) as CUTOFF_DATE FROM ALL_TT_METRICS_2
),
EARLIEST_TO_DELETE AS (
SELECT
    pat_enc_csn_id_coded,
    cath_time,
    emergency_admit_time,
    provider_id,
    case when cath_time >= CUTOFF_DATE then day_since_last_cath else NULL end AS day_since_last_cath
FROM
    ALL_TT_METRICS_2, FIND_EARLIEST_ENC
),
FINAL AS(
SELECT 
    pat_enc_csn_id_coded,
    cath_time,
    emergency_admit_time,
    MIN(day_since_last_cath) AS day_since_last_cath
FROM 
    EARLIEST_TO_DELETE
GROUP BY 
    1,2,3
)

SELECT * FROM FINAL
'''

In [ ]:
dataframe = bqclient.query(query).result().to_dataframe()
dataframe = dataframe[['pat_enc_csn_id_coded', 'day_since_last_cath']]
print(dataframe)
df_list.append(dataframe)

Update: January 9th, 2023 CCR has made edits up to here to change "tpa" to "cath"

### Number of joint TPA cases

In [ ]:
# Create a table of treatment team, do not filter down to docs & nurses.
query_1= '''
CREATE OR REPLACE TABLE `mining-clinical-decisions.rose_team.treatment_team`  AS
WITH TPAP_ACTUAL_TIMES AS
(
SELECT 
    AL.pat_enc_csn_id_coded, 
    min(DATETIME_SUB(AL.cathTime, INTERVAL TMP.JITTER DAY)) as cath_time,
    min(DATETIME_SUB(AL.emergencyAdmitTime, INTERVAL TMP.JITTER DAY)) as emergency_admit_time, 
    min(DATETIME_SUB(DATETIME_SUB(AL.cathTime, INTERVAL TMP.JITTER DAY), INTERVAL 1 HOUR)) as tw_start_time,
    min(DATETIME_ADD(DATETIME_SUB(AL.cathTime, INTERVAL TMP.JITTER DAY), INTERVAL 1 HOUR)) as tw_end_time,

FROM `rose_team.cohort_AL` as AL,
  `rose_team.tmp` as TMP
WHERE AL.rit_uid = TMP.ANON_ID
GROUP BY AL.pat_enc_csn_id_coded
ORDER BY cath_time
),

PROVIDER_MAPPING AS
(
SELECT pat_enc_csn_id_coded, user_deid AS provider_id, emergencyAdmitTime, cathTime, count(*) AS even_count
  FROM `rose_team.cohort_AL` as AL
  WHERE 
    metric_name like "%Lab%" OR metric_name like "% lab%"
    OR metric_name like "%Encounter%" OR metric_name like "%encounter%"
    OR metric_name like "%Flowchart%" OR metric_name like "%flowchart%"
    OR metric_name like "%Order%" OR metric_name like "%order%"   
    OR metric_name like "%Medications%" OR metric_name like "%medications%"
    OR metric_name like "%MAR%" 
    OR metric_name like "%Note%" OR metric_name like "%note%"
    OR metric_name like "%History%" OR metric_name like "%history%"
    OR metric_name like "%Imag%" OR metric_name like "%imag%"
  GROUP BY pat_enc_csn_id_coded, user_deid,emergencyAdmitTime, cathTime
),

TT AS
(
SELECT 
  *
FROM 
  TPAP_ACTUAL_TIMES
INNER JOIN
  PROVIDER_MAPPING
USING 
  (pat_enc_csn_id_coded)
),

  -- Generate AL with actual times
AL_ALL_ACTUAL_TIMES AS
(SELECT AL.*, DATETIME_SUB(AL.access_time_jittered, INTERVAL TMP.JITTER DAY) as access_time_real
FROM `rose_team.shc_access_log_de_dep_id` as AL,
`rose_team.tmp` as TMP
WHERE AL.rit_uid=TMP.ANON_ID
    AND metric_name NOT LIKE 'Inpatient Patient Lists activity accessed'
ORDER BY AL.rit_uid
),

ALL_TT_ENCOUNTERS AS (

SELECT 
  TT.*, access_time_real, metric_name, workstation_id
FROM 
  TT
INNER JOIN
  AL_ALL_ACTUAL_TIMES
ON
  (AL_ALL_ACTUAL_TIMES.user_deid = TT.provider_id)
AND
  access_time_real BETWEEN tw_start_time AND tw_end_time
),

ALL_TT_METRICS AS (

SELECT 
    pat_enc_csn_id_coded,
    cath_time,
    emergency_admit_time,
    tw_start_time, tw_end_time, 
    provider_id AS prov_id, count(*) AS n
FROM 
    ALL_TT_ENCOUNTERS
GROUP BY 1,2,3,4,5,6
ORDER BY emergency_admit_time
)

SELECT * FROM ALL_TT_METRICS
'''



It looks like query_2 does a cartesian product within each index encounter, and then counts pairs of users. I think this double counts users.
Ex: 
pat_enc_csn_id_coded, provider_id
1,1
1,2
1,3

the cartesian product contatenated pair would be:
1,11
1,12
1,13
1,21
1,22
1,23
1,31
1,32
1,33

implying 9 pairings, but really there are only 6
1,11
1,12
1,13
1,22
1,23
1,33

because, eg, 12 & 21 are the same.

The window function count works fine, but I'm rewriting it so that it more explicitly matches UCSF/KPNC code.


In [ ]:
query_2='''
WITH ALL_TT_METRICS AS
(SELECT * FROM `mining-clinical-decisions.rose_team.treatment_team` ),
ALL_TT_METRICS_2 AS (
SELECT 
    DISTINCT
    ALL_TT_METRICS.pat_enc_csn_id_coded,
    ALL_TT_METRICS.cath_time,
    ALL_TT_METRICS.emergency_admit_time,
    case when ALL_TT_METRICS.PROV_ID > ALL_TT_METRICS_COPY.PROV_ID
        then concat(ALL_TT_METRICS.PROV_ID,'_', ALL_TT_METRICS_COPY.PROV_ID) 
        else concat(ALL_TT_METRICS_COPY.PROV_ID,'_', ALL_TT_METRICS.PROV_ID) end AS joint_providers
FROM 
    ALL_TT_METRICS
INNER JOIN 
    ALL_TT_METRICS AS ALL_TT_METRICS_COPY
ON
    ALL_TT_METRICS.pat_enc_csn_id_coded = ALL_TT_METRICS_COPY.pat_enc_csn_id_coded and
    ALL_TT_METRICS.cath_time = ALL_TT_METRICS_COPY.cath_time and
    ALL_TT_METRICS.emergency_admit_time = ALL_TT_METRICS_COPY.emergency_admit_time and 
    ALL_TT_METRICS.PROV_ID != ALL_TT_METRICS_COPY.PROV_ID 
),
SHARED_ENC_COUNT AS (
    SELECT 
    a.pat_enc_csn_id_coded,
    a.cath_time,
    a.emergency_admit_time,
    a.joint_providers,
    count(DISTINCT(b.PAT_ENC_CSN_ID_CODED)) as cath_count_within_6mo
    FROM ALL_TT_METRICS_2 A 
    LEFT join ALL_TT_METRICS_2 B
    on a.joint_providers = b.joint_providers and b.emergency_admit_time 
        between DATETIME_SUB(a.emergency_admit_time, INTERVAL 6 MONTH) and a.emergency_admit_time
    and a.pat_enc_csn_id_coded != b.pat_enc_csn_id_coded
    GROUP BY 
    a.pat_enc_csn_id_coded,
    a.cath_time,
    a.emergency_admit_time,
    a.joint_providers
), WASHOUT_DATE as (
    select DATETIME_ADD(min(cath_time), INTERVAL 6 MONTH) as WASHOUT_DATE from SHARED_ENC_COUNT
), WASHED_JOINT_PROV_LEVEL as (
    select  
    a.pat_enc_csn_id_coded,
    a.cath_time,
    a.emergency_admit_time,
    a.joint_providers,
    case when cath_time < wo.WASHOUT_DATE then NULL else cath_count_within_6mo end as cath_count_within_6mo
    from SHARED_ENC_COUNT a, WASHOUT_DATE wo 
), FINAL as (
SELECT 
    pat_enc_csn_id_coded,
    cath_time,
    emergency_admit_time,
    AVG(cath_count_within_6mo) AS joint_cath_count_within_6mo
FROM 
    WASHED_JOINT_PROV_LEVEL
GROUP BY 
    1,2,3
)
SELECT * FROM FINAL
'''

In [ ]:
_ = bqclient.query(query_1).result().to_dataframe()
print('query 1 executed')

In [ ]:
dataframe = bqclient.query(query_2).result().to_dataframe()
dataframe = dataframe[['pat_enc_csn_id_coded', 'joint_cath_count_within_6mo']]
print(dataframe)
df_list.append(dataframe)

Joint Experience: USING KPNC/UCSF APPROACH

In [ ]:
query_ucsf='''
WITH TT AS
(SELECT * FROM `mining-clinical-decisions.rose_team.treatment_team` ),
  -- Generate AL with actual times
AL_ALL_ACTUAL_TIMES AS
(SELECT AL.*, DATETIME_SUB(AL.access_time_jittered, INTERVAL TMP.JITTER DAY) as access_time_real
FROM `rose_team.shc_access_log_de_dep_id` as AL,
`rose_team.tmp` as TMP
WHERE AL.rit_uid=TMP.ANON_ID
    AND metric_name NOT LIKE 'Inpatient Patient Lists activity accessed'
ORDER BY AL.rit_uid
),
ALL_TT_AL_DATA AS (
SELECT 
  TT.*, csn, access_time_real, metric_name, workstation_id
FROM 
  TT
INNER JOIN
  AL_ALL_ACTUAL_TIMES
ON
  (AL_ALL_ACTUAL_TIMES.user_deid = TT.prov_id)
AND
  access_time_real BETWEEN tw_start_time AND tw_end_time
),
-- FIND ANY COMBO OF USERS WHO SHARE A CSN IN THE AUDIT LOG DATA
WINDOWS_2 AS
( SELECT DISTINCT PAT_ENC_CSN_ID_CODED, datetime_sub(CATH_TIME, INTERVAL 6 MONTH) as WINDOW_START,
CATH_TIME as WINDOW_END from TT),
WINDOW_ENCS AS 
(SELECT DISTINCT 
    WINDOWS_2.PAT_ENC_CSN_ID_CODED,
    ALL_TT_AL_DATA.PROV_ID,
    ALL_TT_AL_DATA.CSN
    FROM WINDOWS_2, ALL_TT_AL_DATA
    where ALL_TT_AL_DATA.access_time_real between WINDOWS_2.WINDOW_START and WINDOWS_2.WINDOW_END
    ),
-- ASSIGN SHARED ENCOUNTERS TO A GIVEN INDEX ENCOUNTER
WINDOW_ENCS_TT as (
    SELECT 
        a.pat_enc_csn_id_coded, a.prov_id, a.CSN 
    FROM WINDOW_ENCS a 
    INNER JOIN TT b 
    on a.PROV_ID = b.PROV_ID and a.PAT_ENC_CSN_ID_CODED = b.PAT_ENC_CSN_ID_CODED
    where a.CSN is not null
),
MUTUAL_ENCS as (
    SELECT 
    DISTINCT 
        a.pat_enc_csn_id_coded, a.CSN,
        case when a.PROV_ID > b.PROV_ID
            then concat(a.PROV_ID, '_', b.PROV_ID) 
            else concat(b.PROV_ID, '_', a.PROV_ID) end as joint_providers
    FROM WINDOW_ENCS_TT a 
    INNER JOIN WINDOW_ENCS_TT b
    on a.csn = b.csn and a.PAT_ENC_CSN_ID_CODED = b.PAT_ENC_CSN_ID_CODED and a.PROV_ID != b.PROV_ID
),
EDGES as (
    SELECT
        PAT_ENC_CSN_ID_CODED, joint_providers,
        count(distinct(case when PAT_ENC_CSN_ID_CODED = CSN then null else CSN end)) as EDGE_WEIGHT
    FROM MUTUAL_ENCS
    GROUP BY 
        PAT_ENC_CSN_ID_CODED, joint_providers
), 
-- For any provider pairs that don't share a CSN, we need to include their edge weights too (which will be 0)
TT_MUTUAL AS (
    SELECT 
    DISTINCT 
        a.pat_enc_csn_id_coded, a.cath_time,
        case when a.PROV_ID > b.PROV_ID
            then concat(a.PROV_ID, '_', b.PROV_ID) 
            else concat(b.PROV_ID, '_', a.PROV_ID) end as joint_providers
    FROM TT a 
    INNER JOIN TT b
    on a.PAT_ENC_CSN_ID_CODED = b.PAT_ENC_CSN_ID_CODED and a.PROV_ID != b.PROV_ID
),
EDGE_TEAM as (
    SELECT 
        a.PAT_ENC_CSN_ID_CODED, a.cath_time, a.joint_providers, 
            coalesce(b.EDGE_WEIGHT, 0) as EDGE_WEIGHT
    FROM TT_MUTUAL a 
    LEFT JOIN EDGES b 
    on a.joint_providers = b.joint_providers and a.PAT_ENC_CSN_ID_CODED = b.PAT_ENC_CSN_ID_CODED 
), 
FINAL as (
    SELECT PAT_ENC_CSN_ID_CODED, CATH_TIME,
        AVG(EDGE_WEIGHT) as CATH_ANY_EXPERIENCE
    from EDGE_TEAM
    GROUP BY 
        PAT_ENC_CSN_ID_CODED, CATH_TIME
        
) 
    SELECT pat_enc_csn_id_coded, 
        case when CATH_TIME < datetime(2011,5,17,0,0,0) then NULL 
            else CATH_ANY_EXPERIENCE end 
            as CATH_ANY_EXPERIENCE 
        FROM FINAL
'''


In [ ]:
dataframe = bqclient.query(query_ucsf).result().to_dataframe()
dataframe = dataframe[['pat_enc_csn_id_coded', 'CATH_ANY_EXPERIENCE']]
print(dataframe)
df_list.append(dataframe)

# Demographics

Update: March 16th, 2022 - Attempting to remove duplicate encounter ID's from the list of encounters. Total coming to 254 versus 244 from other sources. In 5 cases, there are multiple tPA admin times for the same encounter.

When we filtered by treatment team - 10 cases are removed. So presumes that these 10 cases have treatment team size = 0.

Update: April 15th, 2022 - Removed all cases (16 in total) where patient arrived in Stanford ED as a John Doe and so merged MRN limit Audit log measure utility due to missing data.

In [ ]:
query_demo='''
WITH TPAP_ACTUAL_TIMES AS
(
SELECT 
  AL.pat_enc_csn_id_coded, 
  AL.RIT_UID,
  AL.emergencyAdmitTime as ORIG_ED_ADMIT_TIME,
  min(DATETIME_SUB(AL.cathTime, INTERVAL TMP.JITTER DAY)) as cath_time,
  min(DATETIME_SUB(AL.emergencyAdmitTime, INTERVAL TMP.JITTER DAY)) as emergency_admit_time, 
  min(DATETIME_SUB(DATETIME_SUB(AL.cathTime, INTERVAL TMP.JITTER DAY), INTERVAL 1 HOUR)) as tw_start_time,
  min(DATETIME_ADD(DATETIME_SUB(AL.cathTime, INTERVAL TMP.JITTER DAY), INTERVAL 1 HOUR)) as tw_end_time,
FROM `mining-clinical-decisions.rose_team.cohort_AL`as AL,
`mining-clinical-decisions.rose_team.tmp`as TMP
WHERE AL.rit_uid = TMP.ANON_ID
GROUP BY AL.pat_enc_csn_id_coded, AL.rit_uid, AL.emergencyAdmitTime
ORDER BY cath_time
),

PROVIDER_MAPPING AS
(
SELECT pat_enc_csn_id_coded, user_deid AS provider_id, emergencyAdmitTime, cathTime, count(*) AS even_count
  FROM `mining-clinical-decisions.rose_team.cohort_AL` as AL
  WHERE 
    metric_name like "%Lab%" OR metric_name like "% lab%"
    OR metric_name like "%Encounter%" OR metric_name like "%encounter%"
    OR metric_name like "%Flowchart%" OR metric_name like "%flowchart%"
    OR metric_name like "%Order%" OR metric_name like "%order%"   
    OR metric_name like "%Result%" OR metric_name like "%result%" 
    OR metric_name like "%Medications%" OR metric_name like "%medications%"
    OR metric_name like "%MAR%" 
    OR metric_name like "%Note%" OR metric_name like "%note%"
    OR metric_name like "%History%" OR metric_name like "%history%"
    OR metric_name like "%Imag%" OR metric_name like "%imag%"
  GROUP BY pat_enc_csn_id_coded, user_deid, emergencyAdmitTime, cathTime
),
TT AS
( 
  SELECT DISTINCT 
        AL.pat_enc_csn_id_coded,
        al.rit_uid, 
        dm.gender, 
        dm.canonical_race, 
        dm.canonical_ethnicity,
        EXTRACT(YEAR FROM AL.emergency_admit_time) AS year_tpa,
        DATETIME_DIFF(AL.ORIG_ED_ADMIT_TIME, DM.birth_date_jittered, YEAR) AS age,
        DATETIME_DIFF(cath_time,emergency_admit_time, MINUTE) AS cath_minutes
FROM 
  TPAP_ACTUAL_TIMES al
INNER JOIN PROVIDER_MAPPING tm
  ON (al.pat_enc_csn_id_coded = tm.pat_enc_csn_id_coded)
INNER JOIN `mining-clinical-decisions.starr_datalake2018.demographic` AS DM
  ON (DM.rit_uid = AL.rit_uid)
),
demo as (
SELECT DISTINCT 
    pat_enc_csn_id_coded, 
    rit_uid,
    gender,
    canonical_ethnicity as ethnicity,
    case when canonical_race = 'Other' then 'Unknown' else canonical_race end as race,
    case when age between 18 and 49 then '18-49' 
    when age between 50 and 59 then '50-59'
    when age between 60 and 69 then '60-69'
    when age between 70 and 79 then '70-79'
    when age >= 80 then '80+' end as age_range, 
    year_tpa as year
FROM TT
)
SELECT * FROM demo
'''

March 11th - Looks like demo table has encounters that don't match elsewhere due to non-unique encounters

In [ ]:
dataframe = bqclient.query(query_demo).result().to_dataframe()
df_demo = dataframe[['pat_enc_csn_id_coded', 'gender','ethnicity','race','age_range','year']]
print(df_demo)
#df_list.append(dataframe)

In [ ]:
print(dataframe.head())

In [ ]:
print(df_list)

In [ ]:
pd.set_option('display.max_rows', None)

In [ ]:
from functools import reduce
df_merged = reduce(lambda  left,right: pd.merge(left,right,on=['pat_enc_csn_id_coded'],
                                            how='outer'), df_list)
df_merged

In [ ]:
df_merged = df_merged.merge(df_demo, how='inner', on='pat_enc_csn_id_coded')

In [ ]:
df_merged.iloc[[137]]

In [ ]:
df_merged.to_csv('all_metrics.csv')

In [ ]:
import matplotlib.pyplot as plt

plt.matshow(df_merged.corr())
plt.show()

In [ ]:
df_merged.describe()

### Bivariate Coefficients

In [ ]:
from sklearn.linear_model import LinearRegression 
from scipy.stats.stats import pearsonr 

metric_list = ['num_unq_wrkstn', 'avg_task_switching', 'view_count', 'day_since_last_cath', 'joint_cath_count_within_6mo', 'CATH_ANY_EXPERIENCE']
for metric in metric_list:
    X = np.nan_to_num(df_merged[metric].to_numpy().reshape((-1,1)))
    Y = np.nan_to_num(df_merged['cath_min'].to_numpy().reshape((-1,1)))
    #reg = LinearRegression().fit(X, Y)
    X = sm.add_constant(X)
    mod = sm.OLS(Y,X)
    fii = mod.fit()
    print('*********', metric )
    #print('pearson correlation')
    #print(pearsonr(np.reshape(X,(-1,)), np.reshape((Y),(-1,))))
    print('bivariate coefficients')
    print(fii.conf_int(alpha=0.05, cols=None)[1])
    print(fii.params) 
    print((np.mean(X), np.median(X), np.std(X), np.min(X), np.max(X))) 
    print(fii.summary2().tables[1]['P>|t|'].x1)

### Multivariate Coefficients

In [ ]:
# Build contrasts
x_raw = df_merged[['gender','year','age_range','race','ethnicity']]
Y = df_merged['cath_min']
df_dummies = pd.get_dummies(x_raw, prefix=['gender','year','age_range','race','ethnicity'], prefix_sep='.', 
                            columns=['gender','year','age_range','race','ethnicity'])
df_contrasts = df_dummies.drop(['gender.Female', 'year.2017', 'age_range.18-49', 'race.White', 'ethnicity.Non-Hispanic'], axis=1)
model_df = df_contrasts

In [ ]:
df_merged.hist(bins=10, column='joint_cath_count_within_6mo')

In [ ]:
df_merged[df_merged.joint_cath_count_within_6mo > 1]

In [ ]:
metric_list = ['num_unq_wrkstn', 'avg_task_switching', 'view_count', 'day_since_last_cath', 'joint_cath_count_within_6mo', 'CATH_ANY_EXPERIENCE']
for metric in metric_list:
    #X = np.nan_to_num(df_merged[metric].to_numpy().reshape((-1,1)))
    #Y = np.nan_to_num(df_merged['tpa_min'].to_numpy().reshape((-1,1)))
    model_df[metric] = df_merged[metric]
    X = sm.add_constant(model_df)
    mod = sm.OLS(Y,X,missing='drop')
    results = mod.fit()
    print('*********', metric )
    print('multivariate coefficients')
    #print(results.params.loc[metric])
    print(results.summary())
    print(results.conf_int(alpha=0.05, cols=None).loc[metric])
    model_df = model_df.drop(metric,axis=1)

In [ ]:
model_df['joint_cath_count_within_6mo']=np.nan_to_num(df_merged[metric].to_numpy().reshape((-1,1)))
    #Y = np.nan_to_num(df_merged['tpa_min'].to_numpy().reshape((-1,1)))

In [ ]:
model_df

In [ ]:
print(results.summary())

In [ ]:
np.nan_to_num(df_merged['joint_cath_count_within_6mo'].to_numpy().reshape((-1,1)))

 ### Quartile Analysis of Team Variables

In [ ]:
# Build contrasts

# Get raw data
x_raw = df_merged[['joint_cath_count_within_6mo', 'CATH_ANY_EXPERIENCE']]
Y = df_merged['cath_min']

# Convert to categories
x_raw['q_joint_cath'] = pd.qcut(x_raw['joint_cath_count_within_6mo'], 4, labels=["1", "2", "3", "4"])
x_raw['q_any_exp'] = pd.qcut(x_raw['CATH_ANY_EXPERIENCE'], 4, labels=["1", "2", "3", "4"])

# Remove the continuous vars
x_raw = x_raw.drop(['joint_cath_count_within_6mo', 'CATH_ANY_EXPERIENCE'], axis=1)

# Convert to contrasts
df_dummies = pd.get_dummies(x_raw, prefix=['joint', 'any'], prefix_sep='.', 
                            columns=['q_joint_cath','q_any_exp'])
df_contrasts = df_dummies.drop(['joint.1', 'any.1'], axis=1)
qmodel_df = df_contrasts

In [ ]:
metric_list = [['joint.2','joint.3','joint.4'],['any.2','any.3','any.4']]
for metric in metric_list:
    #X = np.nan_to_num(df_merged[metric].to_numpy().reshape((-1,1)))
    X = qmodel_df[metric]
    Y = np.nan_to_num(df_merged['cath_min'].to_numpy().reshape((-1,1)))
    X = sm.add_constant(X)
    mod = sm.OLS(Y,X)
    results = mod.fit()
    print('*********', metric )
    print('quartiles coefficients')
    print(results.params.loc[metric]) 
    print(results.conf_int(alpha=0.05, cols=None).loc[metric])

In [ ]:
print(x_raw)

In [ ]:
print(fii.params)
print(results.params.loc[metric]) #np.mean(X), np.median(X), np.std(X), np.min(X), np.max(X), 
print(results.conf_int(alpha=0.05, cols=None).loc[metric])

In [ ]:
model_df